In [2]:
# importing necessary modules 

import os
import pandas as pd
import csv
import numpy as np
from pathlib import Path


In [3]:
base_folder = Path.cwd()
base_folder

WindowsPath('c:/Users/birap/OneDrive/Desktop/Tradeview/tradingView/Python code for chatGpt')

In [4]:
master_file_path = Path.joinpath(base_folder, "Master File", "CompanyMaster.csv")
directory = Path.joinpath(Path(base_folder).parents[0], "Data Files")
combined_directory = Path.joinpath(Path(base_folder).parents[0], "Delivery Price Combined Files")

input_path =  Path.joinpath(base_folder, "Input File")
output_path = Path.joinpath(base_folder, "Output File")

output_path

WindowsPath('c:/Users/birap/OneDrive/Desktop/Tradeview/tradingView/Python code for chatGpt/Output File')

In [5]:
column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']

non_fno_csv = os.path.join(output_path, '0_non_fno.csv')
fno_csv = os.path.join(output_path, 'fno.csv')
sme_csv = os.path.join(output_path, 'sme_csv.csv')

avgs_file = os.path.join(input_path, 'ShortListed Stocks.xlsx')


In [6]:
def replace_nse_code(row):
    if pd.isna(row['NSE_Code']) or row['NSE_Code'] == "":
        return row['BSE_Code']
    else:
        return row['NSE_Code']
    
def add_nse_prefix(df, column_name):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)

    return df

In [7]:
no_days = 2

file_names = sorted([file for file in os.listdir(combined_directory) if file.lower().endswith(("CSV","csv"))
                        and file.startswith("combined")], reverse=True)[:no_days]

print(file_names)

count = 1

for file_name in file_names:
    
    file_path = os.path.join(combined_directory,file_name)
    csv_file_df = pd.read_csv(file_path)
    
    if count == 1:
        combined_df = csv_file_df
    else:
        combined_df = pd.concat([combined_df, csv_file_df], ignore_index=True)
    
    count += 1
    #print(count)

combined_df['BSE_Code'] = combined_df['BSE_Code'].fillna(0)
combined_df['BSE_Code'] = combined_df['BSE_Code'].astype(int)
combined_df['NSE_Code'] = combined_df['NSE_Code'].fillna("")
combined_df['Company Name'] = combined_df['Company Name'].fillna("")
combined_df['NSE_Code'] = combined_df.apply(replace_nse_code, axis=1)

report_date = file_names[0]
report_date = report_date[9:15]
print(report_date)


['combined_231020.csv', 'combined_231010.csv']
231020


In [8]:
#combined_df.info()


In [9]:
blank_co = combined_df[combined_df['Company Name'] == ""]
#blank_co

In [10]:
dely_value = 50000000

# Assuming your DataFrame is named combined_df
# Sort the DataFrame by Date
combined_df.sort_values(by='Date', inplace=True)

# Create a new DataFrame for the later date and the previous date
later_date = combined_df[combined_df['Date'] == combined_df['Date'].max()]
previous_date = combined_df[combined_df['Date'] == combined_df['Date'].min()]

# Merge the two DataFrames on Company Name, NSE Code, and BSE Code
merged_df = pd.merge(later_date, 
                     previous_date, 
                     on=['Company Name', 'NSE_Code', 'BSE_Code'], 
                     how='outer', 
                     suffixes=('_later', '_previous')
                     )

# Exclude rows where 'Total_Delivery_later' is 0
merged_df = merged_df[merged_df['Total_Delivery_later'] != 0]

# Exclude rows where delivery value today is less than 10000000
merged_df = merged_df[(merged_df['Total_Delivery_later'] * merged_df['Price_later']) >= dely_value]

# Select the desired columns
result_df = merged_df[['Company Name', 'NSE_Code', 'Total_Delivery_later', 'Total_Delivery_previous', 'Price_later', 'Price_previous']].copy()

column_names = ['Company Name','ScripCode','Dely','PrevDel','Price','PrevPrice']

# Rename columns
result_df.columns = column_names

result_df['Dely'] = result_df['Dely'].astype(int)
result_df['PrevDel'] = result_df['PrevDel'].fillna(0)
result_df['PrevDel'] = result_df['PrevDel'].astype(int)


# Calculate the ratio of Total_Delivery_later to Total_Delivery_previous
result_df['DelyRatio'] = result_df['Dely'] / result_df['PrevDel']

# Sort the result DataFrame in descending order based on the ratio
result_df.sort_values(by='DelyRatio', ascending=False, inplace=True)

# Fill NaN values with 0 for cases where there is no previous date
result_df.fillna(0, inplace=True)

# Add a new column 'PriceChange' indicating whether the price went up or down
result_df['PriceChange'] = np.where(result_df['Price'] > result_df['PrevPrice'], 'Up', np.where(result_df['Price'] < result_df['PrevPrice'], 'Down', 'No Change'))


filtered_data = result_df[result_df['DelyRatio'] >= 3]

result_df = filtered_data
# Drop rows where "Company Name" is empty
# Replace empty strings with NaN
result_df['Company Name'].replace('', np.nan, inplace=True)
result_df.dropna(subset=['Company Name'], inplace=True)

result_df.reset_index(drop=True, inplace=True)


# Format the alignment of the 'Company_Name' column to left-align
#result_df.style.set_properties(subset=['Company_Name'], **{'text-align': 'left'})
result_df.style.set_properties(subset=['Company Name'], **{'text-align': 'left'})

formatted_columns = ['Company Name', 'ScripCode','Dely','PrevDel', 'Price', 'PrevPrice','DelyRatio', 'PriceChange']
formatted_data = result_df[formatted_columns].copy()

formatted_data['Dely'] = formatted_data['Dely'].apply(lambda x:f"{int(x):,}")
formatted_data['PrevDel'] = formatted_data['PrevDel'].apply(lambda x:f"{int(x):,}")
formatted_data['DelyRatio'] = formatted_data['DelyRatio'].apply(lambda x:f"{x:.2f}")
formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{round(x, 2):,.2f}")
formatted_data['PrevPrice'] = formatted_data['PrevPrice'].apply(lambda x: f"{round(x, 2):,.2f}")

styled_result = formatted_data.style.set_properties(subset=['Company Name', 'ScripCode'], **{'text-align': 'left'})

# Display the result in a formatted way
#print(styled_result.to_string(index=False))
#print("Delivery comparision over 2 days ", report_date, " value over ", dely_value/100000)
#print(" ")
#print(" ")
#display(styled_result)


In [11]:
result_df.head()

,Company Name,ScripCode,Dely,PrevDel,Price,PrevPrice,DelyRatio,PriceChange
0,Plaza Wires Limited,PLAZACABLE,777816,0,107.35,0.00,inf,Up
1,Supershakti Metaliks Ltd.,541701,150600,0,465.00,0.00,inf,Up
2,Himachal Fibres Ltd.,514010,26918197,7137,14.99,12.83,3771.640325,Up
3,Kintech Renewables Ltd.,512329,9990,25,7751.35,6615.85,399.600000,Up
4,GKW Ltd.,GKWLIMITED,32717,431,1682.20,1009.60,75.909513,Up


In [12]:
columns_to_select = ['Company Name', 'ScripCode', 'Dely', 'PrevDel', 'Price', 'PrevPrice', 'DelyRatio', 'PriceChange']
col_length = [30, 12, 12, 12, 12, 12, 8, 6]
formatted_columns = [col + ' ' * (length - len(col)) for col, length in zip(columns_to_select, col_length)]

def print_formatted_rows(df, columns_to_select):
    print(columns_to_select)
    print(df.head())
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = row[columns_to_select[0]].ljust(35)
        p_cocode = str(row[columns_to_select[1]]).ljust(12)  # Convert to string before formatting
        p_dely = f"{row[columns_to_select[2]]:,.0f}".rjust(12)
        p_prevdely = f"{row[columns_to_select[3]]:,.0f}".rjust(12)  # Correct the typo here
        p_price = f"{row[columns_to_select[4]]:,.2f}".rjust(12)
        p_prevprice = f"{row[columns_to_select[5]]:,.2f}".rjust(12)  # Correct the typo here
        p_delyratio = f"{row[columns_to_select[6]]:,.2f}".rjust(8)  # Correct the typo here
        p_direction = str(row[columns_to_select[7]]).ljust(6)  # Convert to string before formatting

        print(f"{p_coname}{p_cocode}{p_dely}{p_prevdely}{p_price}{p_prevprice}  {p_delyratio}    {p_direction}")

        if i % 5 == 0:
            print()



In [13]:
new_column_names = {
    'Company Name': 'Company Name',
    'ScripCode': 'Symbol',
    'Dely': 'Dely',
    'PrevDel': 'PrevDely',
    'Price': 'Price',
    'PrevPrice': 'PrevPrice',
    'DelyRatio': 'Ratio',
    'PriceChange': 'PriceChange'
}

result_df.rename(columns=new_column_names, inplace=True)

In [14]:
result_df.dtypes

Company Name     object
Symbol           object
Dely              int32
PrevDely          int32
Price           float64
PrevPrice       float64
Ratio           float64
PriceChange      object
dtype: object

In [15]:
# Convert 'Dely' and 'PrevDely' to integers

result_df['Dely'] = result_df['Dely'].astype(int)
result_df['PrevDely'] = result_df['PrevDely'].astype(int)

# Convert 'Price' and 'PrevPrice' to float values with two decimal places
result_df['Price'] = result_df['Price'].astype(float)
result_df['Ratio'] = result_df['Ratio'].astype(float)
result_df['PrevPrice'] = result_df['PrevPrice'].astype(float)


In [16]:
result_df.head()

,Company Name,Symbol,Dely,PrevDely,Price,PrevPrice,Ratio,PriceChange
0,Plaza Wires Limited,PLAZACABLE,777816,0,107.35,0.00,inf,Up
1,Supershakti Metaliks Ltd.,541701,150600,0,465.00,0.00,inf,Up
2,Himachal Fibres Ltd.,514010,26918197,7137,14.99,12.83,3771.640325,Up
3,Kintech Renewables Ltd.,512329,9990,25,7751.35,6615.85,399.600000,Up
4,GKW Ltd.,GKWLIMITED,32717,431,1682.20,1009.60,75.909513,Up


In [17]:
def format_dely(value):
    return '{:,.0f}'.format(value)

# Formatting function for Price and PrevPrice columns
def format_price(value):
    return '{:,.2f}'.format(value)

def format_ratio(value):
    try:
        return '{:.2f}'.format(value)
    except:
        return value

# Apply formatting functions to the specified columns
result_df['Dely'] = result_df['Dely'].apply(format_dely)
result_df['PrevDely'] = result_df['PrevDely'].apply(format_dely)
result_df['Price'] = result_df['Price'].apply(format_price)
result_df['Ratio'] = result_df['Ratio'].apply(format_ratio)
result_df['PrevPrice'] = result_df['PrevPrice'].apply(format_price)



In [24]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph, Spacer
from pathlib import Path
from reportlab.pdfbase.pdfmetrics import registerFontFamily
registerFontFamily('Vera',normal='Vera',bold='VeraBd',italic='VeraIt',boldItalic='VeraBI')

dfs = [result_df]
report_titles = [f"Delivery comparision over 2 days, value over 500, ratio over 3x {report_date}"]

report_prefix_name = f"56-script-report_{str(report_date)}.pdf"
report_path = Path.joinpath(Path(base_folder).parents[0], "Reports", report_prefix_name)

# Define the style for column headers without grid lines
column_header_style = [('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                       ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                       ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                       ]

def create_table_with_header(df, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()
    
    for i in range(6, len(table_data), 6):
        table_data.insert(i, [''] * len(df.columns))
        
    style = [
    ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
    ('ALIGN', (1, 0), (1, -1), 'LEFT'),  # Left align second column
    ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align last column
    ('ALIGN', (2, 0), (-2, -1), 'RIGHT'),  # Right align remaining columns (3rd to second-to-last)
    ]

    t = Table(table_data, repeatRows=1, colWidths=[170, 80, 70, 70, 60, 60, 60, 70])
    # repeatRows=1 to add first column to all the pages
    t.setStyle(column_header_style)
    t.setStyle(style)
    
    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])      # Hide grid lines
    
    return t

def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right
    
    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

left_margin = 20
right_margin = 20
top_margin = 5
bottom_margin = 20  # Add a bottom margin
page_width = letter[0] + 40  # Get width from letter page size
page_height = letter[1] - top_margin - bottom_margin  # Adjust height for margins

# Create a SimpleDocTemplate with adjusted page size and margins
doc = SimpleDocTemplate(
    str(report_path),
    pagesize=(page_width, page_height),
    leftMargin=left_margin,
    rightMargin=right_margin,
    topMargin=top_margin,
    bottomMargin=bottom_margin
)

# Create a list to hold all elements
elements = []
# Iterate through your dataframes and add tables to the content frame
for i, df in enumerate(dfs):
    title = Paragraph(report_titles[i], getSampleStyleSheet()['Title'])

    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 10
    title = Paragraph(report_titles[i], style=h2_style)
    table_width = doc.width - doc.leftMargin - doc.rightMargin
    table = create_table_with_header(dfs[i], add_column_names=True, table_width=table_width)

    # Add elements for title and table
    records_per_page = 30
    chunks = [dfs[i][j:j + records_per_page] for j in range(0, len(dfs[i]), records_per_page)]
    for chunk in chunks:
        elements.extend([title, create_table_with_header(chunk, add_column_names=True, table_width=table_width), PageBreak()])


# Assign the function to be called for each page
doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)


In [27]:
result_df.head()

,Company Name,Symbol,Dely,PrevDely,Price,PrevPrice,Ratio,PriceChange
0,Plaza Wires Limited,PLAZACABLE,"777,816",0,107.35,0.00,inf,Up
1,Supershakti Metaliks Ltd.,541701,"150,600",0,465.00,0.00,inf,Up
2,Himachal Fibres Ltd.,514010,"26,918,197","7,137",14.99,12.83,3771.64,Up
3,Kintech Renewables Ltd.,512329,"9,990",25,"7,751.35","6,615.85",399.60,Up
4,GKW Ltd.,GKWLIMITED,"32,717",431,"1,682.20","1,009.60",75.91,Up


In [28]:
result_df.head()

,Company Name,Symbol,Dely,PrevDely,Price,PrevPrice,Ratio,PriceChange
0,Plaza Wires Limited,PLAZACABLE,"777,816",0,107.35,0.00,inf,Up
1,Supershakti Metaliks Ltd.,541701,"150,600",0,465.00,0.00,inf,Up
2,Himachal Fibres Ltd.,514010,"26,918,197","7,137",14.99,12.83,3771.64,Up
3,Kintech Renewables Ltd.,512329,"9,990",25,"7,751.35","6,615.85",399.60,Up
4,GKW Ltd.,GKWLIMITED,"32,717",431,"1,682.20","1,009.60",75.91,Up


In [29]:
volume_3x = os.path.join(output_path, f'_{report_date}_0_volume_3x_wl.csv')

volume_df_wl = result_df[['Symbol']]

volume_df_nse_wl = add_nse_prefix(volume_df_wl,'Symbol')

volume_df_nse_wl.to_csv(volume_3x, index=False)

In [30]:
numeric_code_df = result_df[result_df['Symbol'].apply(lambda x: isinstance(x, (int, float)))]

print(numeric_code_df[['Symbol', 'Company Name', 'Price', 'Dely', 'Ratio']])


     Symbol               Company Name     Price        Dely    Ratio
1    541701  Supershakti Metaliks Ltd.    465.00     150,600      inf
2    514010       Himachal Fibres Ltd.     14.99  26,918,197  3771.64
3    512329    Kintech Renewables Ltd.  7,751.35       9,990   399.60
36   512068     Deccan Gold Mines Ltd.    130.68     927,240    11.10
54   532468         Kama Holdings Ltd.  3,347.65      24,178     8.21
114  512379   Cressanda Solutions Ltd.     23.60   5,057,983     4.01
